 Definição:

[Documentação](https://python.langchain.com/docs/concepts/output_parsers/)

Os analisadores de saída são responsáveis ​​por pegar a saída de um LLM e transformá-la em um formato mais adequado. Isso é muito útil quando você está usando LLMs para gerar qualquer forma de dados estruturados.

Como sabemos os modelos LLMs podem gerar respostas diferentes para uma mesma pergunta, ou seja, eles não são determinísticos. Por isso, os OutputParsers são componentes do langchain que formatam a saída do modelos em uma estrutura predefinida, seja uma simples string da resposta ou um formato JSON, por exemplo.

Além de ter uma grande coleção de diferentes tipos de analisadores de saída, um benefício diferenciado do LangChain OutputParsers é que muitos deles oferecem suporte a streaming (ou seja, funcionam com a transmissão de dados token a token).

Outro ponto fundamental é que os OutputParsers implementam interface `Runnables`, ou seja,  são componentes de LangChain e implementam as funções `invoke`, `ainvoke` etc. Sua entrada pode ser uma string, ou um `BaseMessage` (mensagem obtida ao invocar um modelo usando `ChatPromptTemplate`).

### Contextualização

Até o momento nós capturávamos as respostas do modelo dentro de uma chain acessando o conteúdo da `AIMessage` resposta. Porém há uma forma mais 'bonita' de se obter a resposta, utilizando um analisador de saída do tipo string:

In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage

# Importa conectando com o cliente OpenAI
from helpers.llm import initialize_llm, logger, pretty_print

# Parte 1 - Importando os componentes do LangChain
llm, _, _ = initialize_llm()

In [ ]:

prompt_template = ChatPromptTemplate([("user", "Escreva um poema em {lingua} sobre o tema: {assunto}")])

# PART 2: Criando a chain
chain1 = prompt_template | llm

# PART 3: Invoke da chain passando as variáveis.
resposta = chain1.invoke({"lingua": "pt-br", "assunto":"frutas"})

pretty_print(resposta)

Nesse caso a resposta será um componente `AIMessage` e estamos acessando o conteúdo da resposta por meio de `.content`. Porém você pode fazer a seguinte combinação:

In [ ]:
from langchain_core.output_parsers import StrOutputParser

analisador_saida = StrOutputParser()

# PART 2: Criando a chain
chain2 = prompt_template | llm |analisador_saida
# PART 3: Invoke da chain passando as variáveis.
resposta = chain2.invoke({"lingua": "pt-br", "assunto":"frutas"})

print(resposta)



Ao executar, você obterá em 'resposta' o conteúdo gerado pelo LLM sem necessitar realizar o acesso pela variável `content`.  

### PydanticOutputParser
[Documentação](https://python.langchain.com/docs/how_to/output_parser_structured/)

Os modelos de linguagem produzem texto. Mas há momentos em que você quer obter informações mais estruturadas do que apenas texto de volta. Embora alguns provedores de modelos suportem maneiras integradas de retornar saída estruturada , nem todos o fazem.

Os analisadores de saída são classes que ajudam a estruturar respostas de modelos de linguagem.

No caso do tipo PydanticOutputParser, é implementada uma saída estruturada usando a classe Pydantic, onde você declara quais variáveis deve ter na saída do LLM para criar uma instancia da sua classe pydantic. Exemplo:

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser  
from langchain.prompts import ChatPromptTemplate  
from pydantic import BaseModel, Field  
   
  
# Definindo a minha estrutura de saída  
class Rota(BaseModel):  
    escolha: int = Field(description="Rota escolhida")  
    pensamento: str = Field(description="Campo para o pensamento que levou a decisão da rota escolhida")  
  
  
# Criando o analisador de saída  
parser = PydanticOutputParser(pydantic_object=Rota)  
  
prompt_template = ChatPromptTemplate([("system",  
                                       "Se a pergunta do usuário for relacionado ao setor financeiro, a escolha deve ser 1, caso contrário a escolha pode ser qualquer numero diferente de 1. \n{format_instructions}\n Pergunta Usuário: {pergunta_user}")],  
                                     partial_variables={"format_instructions": parser.get_format_instructions()})  
  
  

prompt_and_model = prompt_template | llm  
output = prompt_and_model.invoke({"pergunta_user": "Me diga quanto está o dollar."})  
pretty_print(output)

Veja que ele formatou a resposta entre a notação `json`. Dessa forma, é possivel agora encadear na chain o analisador de saida para que tenhamos como resposta uma instancia de `Rota`, ou seja, realizando a adaptação na chain encadeando o `parser`:

In [ ]:
prompt_and_model = prompt_template | llm | parser
output = prompt_and_model.invoke({"pergunta_user": "Me diga quanto está o dollar."})  
print(output)


### JsonOutputParser
[Documentação](https://python.langchain.com/docs/how_to/output_parser_json/)

O `JsonOutputParser` é uma opção interna para solicitar e então analisar a saída JSON de um modelo LLM. 

Pode ser implementado usando Pydantic:


In [ ]:
from langchain_core.output_parsers import JsonOutputParser  
from langchain.prompts import ChatPromptTemplate   
from pydantic import BaseModel, Field  
   
# Definindo a minha estrutura de saída  
class Rota(BaseModel):  
    escolha: int = Field(description="Rota escolhida")  
    pensamento: str = Field(description="Campo para o pensamento que levou a decisão da rota escolhida")  
  
  
# Criando o analisador de saída  
parser = JsonOutputParser(pydantic_object=Rota)  
  
prompt_template = ChatPromptTemplate([("system",  
                                       "Se a pergunta do usuário for relacionado ao setor financeiro, a escolha deve ser 1, caso contrário a escolha pode ser qualquer numero diferente de 1. \n{format_instructions}\n Pergunta Usuário: {pergunta_user}")],  
                                     partial_variables={"format_instructions": parser.get_format_instructions()})  
  
  

prompt_and_model = prompt_template | llm  | parser

output = prompt_and_model.invoke({"pergunta_user": "Me diga quanto está o dollar."})  
print(output)

Ou não usando Pydantic:

*Atenção: Você também pode usar o `JsonOutputParser`sem Pydantic. Isso fará com que o modelo retorne JSON, mas não fornece detalhes sobre qual deve ser o esquema. Ou seja, pode ser que o modelo erre a estrutura do JSON esperada.*

In [ ]:
from langchain_core.output_parsers import JsonOutputParser  
from langchain.prompts import ChatPromptTemplate  
   
  
# Criando o analisador de saída  
parser = JsonOutputParser()
  
prompt_template = ChatPromptTemplate([("system",  
                                       "Se a pergunta do usuário for relacionado ao setor financeiro, a escolha deve ser 1, caso contrário a escolha pode ser qualquer numero diferente de 1. \n{format_instructions}\n Pergunta Usuário: {pergunta_user}")],  
                                     partial_variables={"format_instructions": parser.get_format_instructions()})  
  
  

prompt_and_model = prompt_template | llm  | parser

output = prompt_and_model.invoke({"pergunta_user": "Me diga quanto está o dollar."})  
print(output)

### Outros:

- XMLOutputParser: [Documentação](https://python.langchain.com/docs/how_to/output_parser_xml/)
- YamlOutputParser: [Documentação](https://python.langchain.com/docs/how_to/output_parser_yaml/)
- DatetimeOutputParser:[Documentação](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.datetime.DatetimeOutputParser.html#langchain.output_parsers.datetime.DatetimeOutputParser)
- EnumOutputParser: [Documentação](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.enum.EnumOutputParser.html#langchain.output_parsers.enum.EnumOutputParser)
- PandasDataFrameOutputParser: [Documentação](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.pandas_dataframe.PandasDataFrameOutputParser.html#langchain.output_parsers.pandas_dataframe.PandasDataFrameOutputParser)